In [1]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample
from cmath import nan
import json

In [46]:
#define the relevant path to the current batch you wish to parse
gap_name='pilot_24_hours' #dont change this as this parsing sciprt only handles the same day batches.  

project_name='multicategory'
experiment_name='experiment 1'
batch_name='batch 1'

PATH_TO_BATCH=path.Path.cwd().parent.parent.parent / 'projects' / project_name / experiment_name / 'collected data' / gap_name / batch_name
PATH_TO_BATCH_DATA = PATH_TO_BATCH / 'data'

if PATH_TO_BATCH.exists():
    print('batch folder found at:\n' ,PATH_TO_BATCH)
else: 
    print('path to batch is non existent:\n',PATH_TO_BATCH)

batch folder found at:
 c:\Users\User\Desktop\visual_memory_project\visual-memory-project\projects\birds\experiment 1\collected data\pilot_24_hours\batch 1


In [47]:
qualification_name_for_testin='UPDATE-completed encoding multicategory memory test'  #note that the -space- after the title is improtant as for some reason this is how the qualification name is defined
qualification_name_for_entire_experiment='UPDATE-completed multicategory memory test'

In [48]:
#this section loads the shelf dict in the state it is after finishing the testing session for this batch. 
shelf_dict_after_encoding_name=PATH_TO_BATCH / 'shelf after encoding session closed.txt' #define the name of the relevant shelf for this stage
with open(shelf_dict_after_encoding_name) as f:
    data = f.read()
shelf_dict = json.loads(data)
shelf_dict

{'A3BU8UL4W258UU': [0, 1672259949355],
 'A2YC6PEMIRSOAA': [1, 1672259992394, 1672260336538],
 'A2IQ0QCTQ3KWLT': [2, 1672259962573],
 'APVZGZM1RA3AZ': [3, 1672259965919],
 'A27PVIL93ZMY46': [4, 1672259976932],
 'A11F3MA5FWH6SJ': [5, 1672259987509],
 'A2PPTQXFVA9H38': [6, 1672259987861],
 'A1FJGB5JLZ81XB': [7, 1672260025367],
 'AYWHJVYLOA2B5': [8, 1672260029473],
 'A9YPM325MKYLO': [9, 1672260039075],
 'AGZ3QJFK1BR8V': [10, 1672260111857],
 'AW9T591E49DCX': [11, 1672260117172],
 'A6M7YIG6KKHPA': [12, 1672260301475],
 'A1TKZIBGTP7FE8': [13, 1672260321296],
 'A3HZFB2JLF3JMY': [14, 1672260345507],
 'ARR9GKAG3JFE5': [15, 1672260410283],
 'A1PHDT66U6IK4Q': [16, 1672260558724],
 'A3SKE5B27HLVO6': [17, 1672260574773],
 'A1QT3YA7G8MO63': [18, 1672260449206],
 'A2VLTSW6CXIUMR': [19, 1672260855599],
 'A1FMVUYV72MUO3': [20, 1672260889474],
 'A2376IXNKHYB22': [999, 999, 999],
 'A2XDWB9NVYI3LQ': [22, 1672260987435],
 'AWX3PLN2FS0SW': [999, 999, 999],
 'A2IP3ZAFYGV8M9': [23, 1672261263996],
 'A1QUQ0TV9

In [49]:
#this section reads all the relevant files: 
if (PATH_TO_BATCH / 'Batch_encoding_batch_results.csv').exists():
    encoding_df=pd.read_csv(PATH_TO_BATCH / 'Batch_encoding_batch_results.csv')
else: 
    print(f'no "Batch_encoding_batch_results.csv" was found in data')

if (PATH_TO_BATCH / 'Batch_testing_batch_results.csv').exists():
    testing_df=pd.read_csv(PATH_TO_BATCH / 'Batch_testing_batch_results.csv')
else: 
    print(f'no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)')
if (PATH_TO_BATCH / 'Batch_workers.csv').exists():
    workers_df=pd.read_csv(PATH_TO_BATCH / 'Batch_workers.csv')
else:
    print('No AMAZON worker list was found: please make sure to upload one and name it correctly')



no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)


In [50]:
def find_largest_consequtive_repetition(responses_array):
#add a stop value so the loop will use the logic also if the end of the responses is all repeating: 
    responses_array=np.append(responses_array,[99])
    longest_rep_dict=dict()
    last_input=responses_array[0]
    consequtive_rep_counter=0
    for response in responses_array[1:]: 
        if response==last_input: #if consequtive rep: 
            consequtive_rep_counter=consequtive_rep_counter+1 #add to counter
        else: #once a new entry is in the vector - store the counter rep information 
            if last_input in longest_rep_dict.keys(): #if it was allready stored in our dictionary
                if longest_rep_dict[last_input]<consequtive_rep_counter: #replace the counter only if it is higher than what stored in dict
                    longest_rep_dict[last_input]=consequtive_rep_counter
            else: #this entry is not yet in the dictionary: add it 
                longest_rep_dict[last_input]=consequtive_rep_counter
            
            consequtive_rep_counter=0
            last_input=response
    return max(longest_rep_dict.values())

In [51]:
def process_worker_results(PATH_TO_BATCH_DATA,subject_name):
    cur_sub=pd.read_csv(PATH_TO_BATCH_DATA / subject_name)
    sub_demographics=cur_sub[['workID','Age','Gender']].iloc[0]

    #extract demo related information: 

    #if participant did not click on the arrow during the demo a 'demo_encoding_response.rt' colmumn wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('demo_encoding_response.rt' in cur_sub.columns):
        cur_sub['demo_encoding_response.rt']=np.nan
    demo_columns=['demo_encoding_loop.thisTrialN','DemoImage','DemoCorrect','demo_encoding_response.rt','demo_encoding_response.keys']

    sub_demo_information=cur_sub[demo_columns]
    empty_inds=sub_demo_information.loc[sub_demo_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_information=sub_demo_information.drop(index=empty_inds)


    #extract the demo test columns: 
    demo_test_columns=['demo_test_response.keys','demo_test_response.corr','demo_test_response.rt','demo_test_loop.thisTrialN','DemoImage1','DemoImage2','DemoCorrectTest','demo_bar_2.response']
    sub_demo_test_information=cur_sub[demo_test_columns]
    empty_inds=sub_demo_test_information.loc[sub_demo_test_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_test_information=sub_demo_test_information.drop(index=empty_inds)

    demo_df=pd.concat([sub_demo_information.reset_index(),sub_demo_test_information.reset_index()],axis=1)

    #extract real experiment related information: 

    #if participant did not click on the arrow during the encoding a 'test_encoding_response.rt' column wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('test_encoding_response.rt' in cur_sub.columns):
        cur_sub['test_encoding_response.rt']=np.nan


    encoding_related_columns=['test_encoding_response.keys','test_encoding_response.corr','trials.thisTrialN','target','pair','layer','correct','test_encoding_response.rt','key_resp_end.keys']
    sub_encoding_information=cur_sub[encoding_related_columns]
    #encoding section ends with a key press of the space key: so seperate this phase by finding this space key row
    end_of_section_ind=np.where(sub_encoding_information['key_resp_end.keys']=='space')[0][0]
    sub_encoding_information=sub_encoding_information.iloc[0:end_of_section_ind]
    #remove all the rows that precede the real encoding phase: 
    empty_inds=sub_encoding_information.loc[sub_encoding_information.isnull().apply(lambda x: all(x), axis=1)].index
    sub_encoding_information=sub_encoding_information.drop(index=empty_inds).reset_index()

    #extract real experiment TEST related information: 
    #test_related_columns=['layer','correct','test_test_response.keys','test_test_response.corr','test_test_response.rt','trials_2.thisRepN','trials_2.thisTrialN','trials_2.thisN','trials_2.thisIndex','trials_2.ran','image1','image2']
    #sub_test_information=cur_sub[test_related_columns].iloc[end_of_section_ind+2:-1]


    subject_dictionary=dict()
    subject_dictionary['demographics']=sub_demographics
    subject_dictionary['demo_df']=demo_df
    subject_dictionary['encoding_df']=sub_encoding_information
    #subject_dictionary['test_df']=sub_test_information

    return subject_dictionary



In [52]:

#get all results with Encoding information - extract the arrow attention check accuracy and RT (RT is currently not usd as a criterion)

#this section extract the list of participants from the downloaded results files (and not via the workers or session list csvs) 
# - it will create the qualification_df (a table with information on the worker ids and encoding behavior of all participants that we have files for)
all_filenames=[file for file in PATH_TO_BATCH_DATA.iterdir() if 'csv' in file.name and 'ENCODING' in file.name]
#print(f'current csv files:\n{all_filenames}')

qualification_for_test_df=pd.DataFrame(columns=['workerID','arrow_acc','mean_arrow_RT','demo_rating_var'])
for subject_csv in all_filenames:
    subject_dict=process_worker_results(PATH_TO_BATCH_DATA,path.Path(subject_csv))
    cur_sub_encoding=subject_dict['encoding_df']
    cur_sub_demo = subject_dict['demo_df']

    sname=subject_csv.name.split('_')[1] #change according to actuall format. 

    # demo rating var : 
    rating_std = cur_sub_demo['demo_bar_2.response'].std()
    #get only attention check related info: 
    cur_sub_encoding=cur_sub_encoding[['test_encoding_response.keys','correct','test_encoding_response.rt']].copy().dropna(how = 'all')
    if len(cur_sub_encoding)==0: #empty - the participant didnt respond on the arrow at all
        arrow_acc=0
        RT=nan
    else: 
        RT=cur_sub_encoding['test_encoding_response.rt'].mean()
        arrow_acc=(cur_sub_encoding['correct']==cur_sub_encoding['test_encoding_response.keys']).mean()
    sub_df=pd.DataFrame({'workerID':sname,'arrow_acc':arrow_acc,'mean_arrow_RT':RT,'demo_rating_var':rating_std},index=[sname])
    qualification_for_test_df=pd.concat([qualification_for_test_df,sub_df],axis=0)

#the following part update the qualification_df with information on wether the participant id exists in the amazon workers list: 

#change participants qualifications if they exists in the workers list based on thier encoding arrow accuracy
qualification_for_test_df['in_encoding_workers_list']=nan

for curr_worker_ID in qualification_for_test_df.index:
    if curr_worker_ID in encoding_df['WorkerId'].values:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(1)
    else:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(0)


## the section below seperate between 4 types of participants: 
    * qualified and in the workers list
    * qualified and not in the workers list 
    * disqualified and in the worker list
    * disqualified and not in the worker list

In [ ]:
qualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==1)& (qualification_for_test_df['demo_rating_var']>0),:].copy()
print('qualified participants')
qualified_df

In [54]:
qualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==0)&(qualification_for_test_df['demo_rating_var']>0),:].copy()
print('qualified participants that are not in the workers list (and thus we cant add a qualification for them)')
qualified_df_not_in_workers_list

qualified participants that are not in the workers list (and thus we cant add a qualification for them)


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A2YC6PEMIRSOAA,A2YC6PEMIRSOAA,1.0,0.82822,0.0


In [55]:
disqualified_df_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('disqualified participants that are in the workers list:')
disqualified_df_in_workers_list

disqualified participants that are in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A39M9PZLH3J1NF,A39M9PZLH3J1NF,0.0,NaN,1.0
AYWHJVYLOA2B5,AYWHJVYLOA2B5,0.4,0.03335,1.0


In [56]:
disqualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('disqualified participants that are NOT in the workers list:')
disqualified_df_not_in_workers_list

disqualified participants that are NOT in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A16JG2M1SEGWPW,A16JG2M1SEGWPW,0.4,2.5517,0.0


In [57]:
qualified_df['type']='qualified in workers list'
qualified_df_not_in_workers_list['type']='qualified not in workers list'
disqualified_df_in_workers_list['type']='disqualified in workers list'
disqualified_df_not_in_workers_list['type']='disqualified not in workers list'
info_from_all_downloaded_datafiles=pd.concat([qualified_df,qualified_df_not_in_workers_list,disqualified_df_in_workers_list,disqualified_df_not_in_workers_list],axis=0)
info_summary_df=info_from_all_downloaded_datafiles.groupby('type').aggregate({'arrow_acc':'count'})
info_summary_df['worker_ids']=nan
info_summary_df['worker_ids']=info_summary_df['worker_ids'].astype(object)
if 'qualified in workers list' in info_summary_df.index: 
    info_summary_df.at['qualified in workers list','worker_ids']=qualified_df.index.to_list()
if 'qualified not in workers list' in info_summary_df.index:    
    info_summary_df.at['qualified not in workers list','worker_ids']=qualified_df_not_in_workers_list.index.to_list()
if 'disqualified in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified in workers list','worker_ids']=disqualified_df_in_workers_list.index.to_list()
if 'disqualified not in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified not in workers list','worker_ids']=disqualified_df_not_in_workers_list.index.to_list()

info_summary_df

,arrow_acc,worker_ids
type,,
disqualified in workers list,2,"[A39M9PZLH3J1NF, AYWHJVYLOA2B5]"
disqualified not in workers list,1,[A16JG2M1SEGWPW]
qualified in workers list,37,"[A11F3MA5FWH6SJ, A11J9UQ036KJA1, A1DZMZTXWOM9M..."
qualified not in workers list,1,[A2YC6PEMIRSOAA]


In [58]:
#update the workers list with the relevant qualification: 
new_shelf_dict=shelf_dict.copy()

#add the qualification for all eligible participants for the test session: 
ind_in_workers_df=[]
print('\nupdating qualifications for participants elligable for test session\n',list(qualified_df.index))
for worker_id in qualified_df.index:
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin].index[0])

print('\nupdating qualifications for participants elligable for test session that are not in the workers list\n',list(qualified_df_not_in_workers_list.index))
for worker_id in qualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#remove from experiment all those that disqualified (so they wont be able to take the test session): 
print('\nupdating qualifications for participants that were disqualified for test session and are in the workers list\n',list(disqualified_df_in_workers_list.index))
for worker_id in disqualified_df_in_workers_list.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment].index[0])
        
        if worker_id in shelf_dict.keys(): #change thier shelf entree (so they wont waste a valid csv number)
            prev_entriees=shelf_dict[worker_id]
            cur_entriees=prev_entriees.copy()
            cur_entriees[0]=999
            new_shelf_dict[worker_id]=cur_entriees
            print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#handle those that disqualified but are not in the amazon worker list: (change thier shelf entries to 999)
print('\nhandling participants that exists in the shelf but not in the amazon workers list (reseting thier shelf entriees)\n',list(disqualified_df_not_in_workers_list.index))
for worker_id in disqualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees.copy()
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#print the two relevant columns of the updated workers_list: 
print('these are all the participants that will be updated in the workers list:')
workers_df.loc[ind_in_workers_df,['Worker ID',qualification_name_for_testin,qualification_name_for_entire_experiment]]



updating qualifications for participants elligable for test session
 ['A11F3MA5FWH6SJ', 'A11J9UQ036KJA1', 'A1DZMZTXWOM9MR', 'A1FJGB5JLZ81XB', 'A1FMVUYV72MUO3', 'A1OM5NWYYYJKQW', 'A1PHDT66U6IK4Q', 'A1QT3YA7G8MO63', 'A1SNC8UL8YFRH5', 'A1TKZIBGTP7FE8', 'A1YV7SCB1OHMUT', 'A25PFSORDO3SWQ', 'A27PVIL93ZMY46', 'A2IP3ZAFYGV8M9', 'A2M183CETUMR96', 'A2VLTSW6CXIUMR', 'A2XDWB9NVYI3LQ', 'A2YVQKJX3FO6P', 'A31T2PF4RV3GTF', 'A39VAFCIGP83ER', 'A3ATB5GC4BQIH0', 'A3BU8UL4W258UU', 'A3CH1Z6J9R38G9', 'A3KP8KFGG6734Q', 'A3SKE5B27HLVO6', 'A41APS6V2Z1FJ', 'A6M7YIG6KKHPA', 'A7VA2Y4H6U31O', 'A9YPM325MKYLO', 'AAF1SJ9FCBF75', 'ACGGIBC0P38HU', 'AGZ3QJFK1BR8V', 'AMPR904VJJFZY', 'APVZGZM1RA3AZ', 'ARR9GKAG3JFE5', 'AW9T591E49DCX', 'AXPZAP62ZYWP8']

updating qualifications for participants elligable for test session that are not in the workers list
 ['A2YC6PEMIRSOAA']
changed subject A2YC6PEMIRSOAA previous shelf entriee: [999, 1672259992394, 1672260336538] to [999, 1672259992394, 1672260336538] in the shelf

updating q

,Worker ID,UPDATE-completed encoding successfully,UPDATE-completed birds memory rep
28,A11F3MA5FWH6SJ,1.0,NaN
30,A11J9UQ036KJA1,1.0,NaN
245,A1DZMZTXWOM9MR,1.0,NaN
271,A1FJGB5JLZ81XB,1.0,NaN
274,A1FMVUYV72MUO3,1.0,NaN
426,A1OM5NWYYYJKQW,1.0,NaN
445,A1PHDT66U6IK4Q,1.0,NaN
476,A1QT3YA7G8MO63,1.0,NaN
510,A1SNC8UL8YFRH5,1.0,NaN
526,A1TKZIBGTP7FE8,1.0,NaN


In [59]:
#save the worker list to upload (to update the qualifications) -> this is what you upload to amazon to update the qualifications of relevant participants
workers_df.to_csv(PATH_TO_BATCH/'Batch_workers_for_upload_after_encoding.csv',index=False)

In [60]:
#this section remove participants that were assigned with an unavailble csv from the shelf (also not changing qualifications for them at all so they could come back to another encoding session in the future) 

max_csv_number=62
participants_exceeding_available_csvs=[]
for key in shelf_dict.keys():
    cur_values=shelf_dict[key]
    if (cur_values[0]>max_csv_number) and (not(cur_values[0]>=998)):
        participants_exceeding_available_csvs.append(key)
        if key in new_shelf_dict.keys():
            del new_shelf_dict[key]

print(f'the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of {max_csv_number}:\nthis probably means that they did not even see the encoding of the experiment and should be considered as never participated in our\n{participants_exceeding_available_csvs}')

the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of 62:
this probably means that they did not even see the encoding of the experiment and should be considered as never participated in our
[]


In [61]:
#this section gives 999 to participants that are in the shelf but for some reason are not in the batch_results_file (so they probably saw the experiment, but we dont know how they performed during encoding so we discard them)
for worker_id in encoding_df['WorkerId']:
    if worker_id not in qualification_for_test_df.index:
        change_msg='- is not in the shelf'
        if worker_id in new_shelf_dict.keys():
            cur_values=new_shelf_dict[worker_id]
            new_values=cur_values.copy()
            new_values[0]=999
            new_shelf_dict[worker_id]=new_values
            change_msg=f'- shelf entry changed from: {cur_values} to {new_values}'

        print(f'{worker_id} did not have an encoding file ' + change_msg)

A1Y2W25NF6T431 did not have an encoding file - shelf entry changed from: [33, 1672264261852] to [999, 1672264261852]


In [62]:
#this section find participants that have a valid shelf entree but are not in the worker list (at all) 
#going over the worker list and not the results list because it is possible these participant were entered into the shelf due to a previous batchwoekre
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id not in workers_df['Worker ID'].values:
            new_values=cur_values.copy()
            new_values[0]=999
            change_msg=f' - shelf entry changed from: {cur_values} to {new_values}'
            new_shelf_dict[worker_id]=new_values
            print(f'{worker_id} absent from WORKERS LIST ' + change_msg)


A2PPTQXFVA9H38 absent from WORKERS LIST  - shelf entry changed from: [6, 1672259987861] to [999, 1672259987861]


In [63]:
#this section extracts the names of the remaining valid participants in the shelf: 
valid_participants_in_shelf_for_test=[]
valid_participants_csvnumber_in_shelf_for_test=[]
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id in workers_df['Worker ID'].values:
            valid_participants_in_shelf_for_test.append(worker_id)
            valid_participants_csvnumber_in_shelf_for_test.append(cur_values[0])

valid_shelf_participants_info_df=pd.DataFrame({'Worker ID':valid_participants_in_shelf_for_test,'numeric csv':valid_participants_csvnumber_in_shelf_for_test},index=valid_participants_in_shelf_for_test)
valid_shelf_participants_info_df

,Worker ID,numeric csv
A3BU8UL4W258UU,A3BU8UL4W258UU,0
A2IQ0QCTQ3KWLT,A2IQ0QCTQ3KWLT,2
APVZGZM1RA3AZ,APVZGZM1RA3AZ,3
A27PVIL93ZMY46,A27PVIL93ZMY46,4
A11F3MA5FWH6SJ,A11F3MA5FWH6SJ,5
A1FJGB5JLZ81XB,A1FJGB5JLZ81XB,7
A9YPM325MKYLO,A9YPM325MKYLO,9
AGZ3QJFK1BR8V,AGZ3QJFK1BR8V,10
AW9T591E49DCX,AW9T591E49DCX,11
A6M7YIG6KKHPA,A6M7YIG6KKHPA,12


In [64]:
valid_shelf_participants_info_df.join(qualification_for_test_df)

,Worker ID,numeric csv,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A3BU8UL4W258UU,A3BU8UL4W258UU,0,A3BU8UL4W258UU,1.0,0.492,1.0
A2IQ0QCTQ3KWLT,A2IQ0QCTQ3KWLT,2,NaN,NaN,NaN,NaN
APVZGZM1RA3AZ,APVZGZM1RA3AZ,3,APVZGZM1RA3AZ,1.0,0.81216,1.0
A27PVIL93ZMY46,A27PVIL93ZMY46,4,A27PVIL93ZMY46,1.0,0.7446,1.0
A11F3MA5FWH6SJ,A11F3MA5FWH6SJ,5,A11F3MA5FWH6SJ,1.0,1.20072,1.0
A1FJGB5JLZ81XB,A1FJGB5JLZ81XB,7,A1FJGB5JLZ81XB,1.0,1.53006,1.0
A9YPM325MKYLO,A9YPM325MKYLO,9,A9YPM325MKYLO,1.0,1.25654,1.0
AGZ3QJFK1BR8V,AGZ3QJFK1BR8V,10,AGZ3QJFK1BR8V,1.0,1.1552,1.0
AW9T591E49DCX,AW9T591E49DCX,11,AW9T591E49DCX,0.8,1.025025,1.0
A6M7YIG6KKHPA,A6M7YIG6KKHPA,12,A6M7YIG6KKHPA,1.0,0.8171,1.0


In [65]:
#this is the updated shelf: <<<< copy >>>> from here and place in the shelf location in pavlovia.org
import json
json.dumps(new_shelf_dict)

'{"A3BU8UL4W258UU": [0, 1672259949355], "A2YC6PEMIRSOAA": [999, 1672259992394, 1672260336538], "A2IQ0QCTQ3KWLT": [2, 1672259962573], "APVZGZM1RA3AZ": [3, 1672259965919], "A27PVIL93ZMY46": [4, 1672259976932], "A11F3MA5FWH6SJ": [5, 1672259987509], "A2PPTQXFVA9H38": [999, 1672259987861], "A1FJGB5JLZ81XB": [7, 1672260025367], "AYWHJVYLOA2B5": [999, 1672260029473], "A9YPM325MKYLO": [9, 1672260039075], "AGZ3QJFK1BR8V": [10, 1672260111857], "AW9T591E49DCX": [11, 1672260117172], "A6M7YIG6KKHPA": [12, 1672260301475], "A1TKZIBGTP7FE8": [13, 1672260321296], "A3HZFB2JLF3JMY": [14, 1672260345507], "ARR9GKAG3JFE5": [15, 1672260410283], "A1PHDT66U6IK4Q": [16, 1672260558724], "A3SKE5B27HLVO6": [17, 1672260574773], "A1QT3YA7G8MO63": [18, 1672260449206], "A2VLTSW6CXIUMR": [19, 1672260855599], "A1FMVUYV72MUO3": [20, 1672260889474], "A2376IXNKHYB22": [999, 999, 999], "A2XDWB9NVYI3LQ": [22, 1672260987435], "AWX3PLN2FS0SW": [999, 999, 999], "A2IP3ZAFYGV8M9": [23, 1672261263996], "A1QUQ0TV9KVD4C": [21, 16722